# Add joint WHO-China metadata

In [ ]:
import pandas as pd

metadata = pd.read_csv(snakemake.input.metadata, sep="\t")
assert len(metadata) == metadata["accession"].nunique()

jointWHO_metadata = pd.read_csv(snakemake.input.jointWHO_metadata)
assert len(jointWHO_metadata) == jointWHO_metadata["accession"].nunique()

assert set(jointWHO_metadata["accession"]).issubset(metadata["accession"])

metadata = (
    metadata
    .merge(
        jointWHO_metadata.assign(
            joint_WHO_report_2019_case=lambda x: x["from_market"].map(
                lambda m: "yes - from Huanan Market" if m else "yes - not from Huanan Market"
            )
        )[["accession", "joint_WHO_report_2019_case"]],
        how="left",
        validate="one_to_one",
    )
    .assign(joint_WHO_report_2019_case=lambda x: x["joint_WHO_report_2019_case"].fillna("no"))
)

metadata.to_csv(snakemake.output.metadata, sep="\t", index=False)

metadata